In [9]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', 'jmLbTuDdD101oZhvxhOT')
        .set('spark.hadoop.fs.s3a.secret.key', 'OhhTM9qPzkFWELBaLr5q9zjGOEeg0T7d67twITL6')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson3 USING iceberg PARTITIONED BY (countryOfOriginCode) AS SELECT * FROM csv_open_2023 ORDER BY countryOfOriginCode;").show()

## ALTER PARTITIONING BASED ON TRUNCATE FIRST LETTER OF firstName
spark.sql("ALTER TABLE nessie.df_open_2023_lesson3 ADD PARTITION FIELD truncate(1, firstName)").show()

## REWRITE ALL THE DATA SO IT ALL USES UPDATED PARTITONING
spark.sql("CALL nessie.system.rewrite_data_files('df_open_2023_lesson3')").show()

Spark Running


++
||
++
++

++
||
++
++



ERROR:root:Exception while sending command.                     (53 + 11) / 200]
Traceback (most recent call last):
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o51.sql

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/docker/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [5]:
## Query the iceberg table
spark.sql("Select * from nessie.df_open_2023_lesson3 limit 10").show()

+------------+-------------------+---------+------------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|     competitorName|firstName|    lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-------------------+---------+------------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     1246569|       Kyle Spenard|     Kyle|     Spenard|   ACT|     M|                 BB|           Barbados|      35|North America East|       5295|CrossFit East Woo...| 22| 72 in|185 lb|      10023|       64897|       1|2023|
|      382704|        Kamal Patel|    Kamal|       Patel|   ACT|     M|         

In [7]:
## ALTER PARTITIONING BASED ON TRUNCATE FIRST LETTER OF firstName
spark.sql("ALTER TABLE nessie.df_open_2023_lesson3 ADD PARTITION FIELD truncate(1, firstName)").show()

## REWRITE ALL THE DATA SO IT ALL USES UPDATED PARTITONING
spark.sql("CALL nessie.system.rewrite_data_files(table => 'df_open_2023_lesson3', where countryOfOriginCode = \)").show

IllegalArgumentException: Cannot add duplicate partition field null=truncate[1](ref(name="firstName")), conflicts with 1001: firstName_trunc_1: truncate[1](3)